#Automatic Differentiation

Automatic Differentiation is a core tool used to calculate derivatives which are key to machine learning. This is my personal implementation.

Eric Buehler 2023

In [327]:
#https://towardsdatascience.com/build-your-own-automatic-differentiation-program-6ecd585eec2a
#https://e-dorigatti.github.io/math/deep%20learning/2020/04/07/autodiff.html
#https://jingnanshi.com/blog/autodiff.html

In [328]:
from abc import ABC, abstractmethod
from typing import *
from functools import reduce

In [329]:
class DifferentiableValue(ABC):
    @abstractmethod
    def backward(self, var):
        pass

    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def __repr__(self) -> str:
        pass

In [330]:
class Graph:
    def __init__(self):
        self.values = []
        global _graph
        _graph = self

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        pass

    def forward(self):
        return self.values[0].forward()

    def backward(self, var: DifferentiableValue):
        graph = self.values.copy()
        res = self.values[0].backward(var)
        self.values = graph
        return res

    def get_gradient(self) -> List[Any]:
        return list(filter(lambda item: item != None, self.values[0].get_gradient()))

def remove_copies(inputs = List[DifferentiableValue]):
    for input in inputs:
        for i, value in enumerate(_graph.values):
            if value.id == input.id:
                _graph.values.pop(i)

In [331]:
class Constant(DifferentiableValue):
    count = 0

    def __init__(self, value, name = None):
        _graph.values.append(self)
        self.value = value
        self.name = name
        Constant.count += 1
        self.id = Variable.count
        self.gradient = None
        
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        res = Constant(0)
        res.gradient = 0
        return res
        
    def forward(self) -> Any:
        return self.value

    def get_gradient(self) -> List[Any]:
        return [self.gradient]

    def __repr__(self) -> str:
        return f"Constant('{self.name}' {self.value}, g={self.gradient})"

class Variable(DifferentiableValue):
    count = 0

    def __init__(self, value = None, name = None):
        _graph.values.append(self)
        self._value = value
        self.name = name
        Variable.count += 1
        self.id = Variable.count
        self.gradient = None
        
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        if self == var:
            res = Constant(1)
            res.gradient = 1
            return res
        res = Constant(0)
        res.gradient = 0
        return res
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        if self.value == None:
            raise ValueError("Variable does not have value")
        self._value = value
        
    def forward(self) -> Any:
        if self.value == None:
            raise ValueError("Variable does not have value")
        return self.value

    def get_gradient(self) -> List[Any]:
        return [self.gradient]

    def __repr__(self) -> str:
        return f"Variable('{self.name}' {self.value}, g={self.gradient})"

In [332]:
class Sum(DifferentiableValue):
    count = 0

    def __init__(self, inputs = List[DifferentiableValue]):
        remove_copies(inputs)
        _graph.values.append(self)
        self.inputs = inputs
        Sum.count += 1
        self.id = Variable.count
        self.gradient = None
        
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        res = Sum([input.backward(var) for input in self.inputs])
        res.gradient = res.forward()
        return res
        
    def forward(self) -> Any:
        return sum([input.forward() for input in self.inputs])

    def get_gradient(self) -> List[Any]:
        grad = [self.gradient]
        for input in self.inputs:
            grad.extend(input.get_gradient())
        return list(filter(lambda item: item != None, grad))

    def __repr__(self) -> str:
        return "Sum({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Product(DifferentiableValue):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        remove_copies([left, right])
        _graph.values.append(self)
        self.inputs = [left, right]
        Product.count += 1
        self.id = Variable.count
        self.gradient = None
        
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        res = self.inputs[1].backward(var)*self.inputs[0] + self.inputs[0].backward(var)*self.inputs[1]
        res.gradient = res.forward()
        return res
        
    def forward(self) -> Any:
        return reduce((lambda x, y: x * y), [input.forward() for input in self.inputs])

    def get_gradient(self) -> List[Any]:
        grad = [self.gradient]
        for input in self.inputs:
            grad.extend(input.get_gradient())
        return list(filter(lambda item: item != None, grad))

    def __repr__(self) -> str:
        return "Product({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Power(DifferentiableValue):
    count = 0

    def __init__(self, base: DifferentiableValue, pow: DifferentiableValue):
        remove_copies([base, pow])
        _graph.values.append(self)
        self.inputs = [base, pow]
        Power.count += 1
        self.id = Variable.count
        self.gradient = None
         
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        if self.inputs[0] != var:
            return self.foward()
        res = self.inputs[1] * (self.inputs[0] ** (self.inputs[1] + Constant(-1)*Constant(1)))
        res.gradient = res.forward()
        return res
        
    def forward(self) -> Any:
        return self.inputs[0].forward() ** self.inputs[1].forward()

    def get_gradient(self) -> List[Any]:
        grad = [self.gradient]
        for input in self.inputs:
            grad.extend(input.get_gradient())
        return list(filter(lambda item: item != None, grad))

    def __repr__(self) -> str:
        return "Power({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Quotient(DifferentiableValue):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        remove_copies([left, right])
        _graph.values.append(self)
        self.inputs = [left, right]
        Product.count += 1
        self.id = Variable.count
        self.gradient = None
        
    def backward(self, var: DifferentiableValue) -> DifferentiableValue:
        numerator = self.inputs[1] * self.inputs[0].backward(var) + Constant(-1) * self.inputs[0] * self.inputs[1].backward(var)
        denominator = self.inputs[1] ** Constant(2)
        res = numerator / denominator
        res.gradient = res.forward()
        return res
        
    def forward(self) -> Any:
        return reduce((lambda x, y: x / y), [input.forward() for input in self.inputs])

    def get_gradient(self) -> List[Any]:
        grad = [self.gradient]
        for input in self.inputs:
            grad.extend(input.get_gradient())
        return list(filter(lambda item: item != None, grad))

    def __repr__(self) -> str:
        return "Quotient({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

In [333]:
DifferentiableValue.__add__ = lambda self, other: Sum([self, other])
DifferentiableValue.__mul__ = lambda self, other: Product(self, other)
DifferentiableValue.__truediv__ = lambda self, other: Quotient(self, other)
DifferentiableValue.__pow__ = lambda self, other: Power(self, other)

In [334]:
with Graph() as graph:
    x = Variable(2, "x")
    y = x*Constant(2)

print("Graph:")
print(graph.values)
print()

print("Forward value:")
print(graph.forward())
print()

print("Backward graph:")
print(graph.backward(x))
print()

print("Backward value:")
print(graph.backward(x).forward())

print(graph.backward(x).get_gradient())

del graph

Graph:
[Product(Variable('x' 2, g=None), Constant('None' 2, g=None), g=None)]

Forward value:
4

Backward graph:
Sum(Product(Constant('None' 0, g=0), Variable('x' 2, g=None), g=None), Product(Constant('None' 1, g=1), Constant('None' 2, g=None), g=None), g=2)

Backward value:
2
[2, 0, 1]
